In [10]:
# Initialize a new VAME project
import functions_bondjango as bd
import functions_vame as fv
import paths
import vame
import random
import os
import shutil
import align_demo_DGN

In [7]:
# get the file paths
# define the search string
search_string = 'rig:miniscope, slug:DG_190806_a'

# define the target model
target_model = 'video_experiment'
# get the queryset
files = bd.query_database(target_model, search_string)

# pick a random subset of 20
files = random.sample(files, 20)

# get the video paths in a list
video_paths = [el['avi_path'] for el in files]

# define the working directory
working_directory = paths.vame_path

In [14]:
print(files[0])

{'url': 'http://192.168.236.135:8080/loggers/video_experiment/09_08_2020_15_56_34_miniscope_dg_200617_b_succ_injured/', 'mouse': 'DG_200617_b', 'avi_path': 'J:\\Drago Guggiana Nilo\\Prey_capture\\VideoExperiment\\09_08_2020_15_56_34_miniscope_DG_200617_b_succ_injured.avi', 'bonsai_path': 'J:\\Drago Guggiana Nilo\\Prey_capture\\VideoExperiment\\09_08_2020_15_56_34_miniscope_DG_200617_b_succ_injured.csv', 'date': '2020-09-08T15:56:34Z', 'dlc_path': '', 'experiment_type': ['Free_behavior'], 'fluo_path': '', 'id': 1976, 'imaging': 'doric', 'lighting': 'normal', 'notes': 'injured', 'owner': 'drguggiana', 'preproc_files': [], 'result': 'succ', 'rig': 'miniscope', 'slug': '09_08_2020_15_56_34_miniscope_dg_200617_b_succ_injured', 'sync_path': 'J:\\Drago Guggiana Nilo\\Prey_capture\\VideoExperiment\\09_08_2020_15_56_34_syncMini_DG_200617_b_succ_injured.csv', 'tif_path': 'J:\\Drago Guggiana Nilo\\Prey_capture\\VideoExperiment\\09_08_2020_15_56_34_miniscope_DG_200617_b_succ_injured.tif'}


In [8]:
# initialize the project
config = vame.init_new_project(project='VAME_prey', videos=video_paths, working_directory=working_directory, videotype='.avi')

Created "F:\VAME_projects\VAME_prey-Nov24-2020\videos"
Created "F:\VAME_projects\VAME_prey-Nov24-2020\data"
Created "F:\VAME_projects\VAME_prey-Nov24-2020\results"
Created "F:\VAME_projects\VAME_prey-Nov24-2020\model"
Copying the videos 

A VAME project has been created. 

Next use vame.create_trainset(config) to split your data into a train and test set. 
Afterwards you can use vame.rnn_model() to train the model on your data.


In [12]:
# Align video trajectories

# get the project folder
project_folder, _ = os.path.split(config)

# for all the videos
for el in files:
    # load the dlc file path
    dlc_path = el['dlc_path']
    # get the basename and folder
    dlc_folder, dlc_basename = os.path.split(dlc_path)
    # define the cropping
    crop_size = (0, 0)
    # egocentrically align the data and save
    fv.run_alignment(dlc_folder, project_folder, dlc_basename, '.avi', crop_size


F:\VAME_projects\VAME_prey-Nov24-2020


In [ ]:
# Create the training set

vame.create_trainset(config)

In [ ]:
# Train the network

# remember to set the parameters in the bonfig.yaml

vame.rnn_model(config, model_name='VAME_prey_model', pretrained_weights=False, pretrained_model=None)

In [ ]:
# Evaluate the network

vame.evaluate_model(config, model_name='VAME_prey_model')

In [ ]:
# Segment behavior

vame.behavior_segmentation(config, model_name='VAME_prey_model', cluster_method='kmeans', n_cluster=[30])

In [ ]:
# Quantify behavior

vame.behavior_quantification(config, model_name='VAME_prey_model', cluster_method='kmeans', n_cluster=30)